In [1]:
!pip install mplfinance

^C



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# imports
import pandas as pd
import yfinance as yf
import numpy as np
import mplfinance as mpf

In [29]:
## add dataset to the project
def load_stock_memory(name):
    path = f'../Data/{name}.csv'
    data = pd.read_csv(path, parse_dates=['Date'], index_col='Date') #index_col='Date'
    # data['Date'] = pd.to_datetime(data['Date'])

    data['Prev_Close'] = data.Close.shift(1)
    data = data.iloc[1:]
    return data
def load_stock_yfinance(name,start="2021-01-01", end="2024-02-01", interval="1d"):
    data = yf.download(name, start=start, end=end, interval=interval)
    return data

stock_data = load_stock_yfinance("TRU.NS")
stock_data['pct_change'] = stock_data['Close'].pct_change().rolling(7).mean() *100
stock_data = stock_data.iloc[7:]
stock_data = stock_data[stock_data['Volume'] != 0]
stock_data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,pct_change
Date,,,,,,,
2022-11-21,58.549999,58.799999,56.349998,56.900002,56.890629,113381,-0.827983
2022-11-22,57.900002,57.900002,55.599998,56.049999,56.040764,115515,-1.112346
2022-11-23,57.400002,57.400002,55.099998,55.799999,55.790806,128608,-0.764203
2022-11-24,55.799999,58.400002,54.849998,56.650002,56.640667,187303,-0.255785
2022-11-25,57.450001,57.900002,56.150002,56.400002,56.390709,58550,-0.294092


In [9]:
# getRelativeValues return the relative of High, Close, Low for a given index i and stock_data
def getRelativeValues(i, stock_data):
    # use the index i to get the High, Close, Low values
    high = stock_data.iloc[i]['High']
    close = stock_data.iloc[i]['Close']
    low = stock_data.iloc[i]['Low']
    op = stock_data.iloc[i]['Open']
    return high/op , close/op, low/op

# calculate relative midpoints based on index i and j
def calculateMidpoints(i, j, k, stock_data):
    # get the relative values of High, Close, Low for index i and j
    # high_i, _, low_i = getRelativeValues(i, stock_data)
    # high_j, _, low_j = getRelativeValues(j, stock_data)
    # # calculate relative midpoints
    # mi = ((high_i - low_i) / 2) + low_i
    # mj = ((high_j - low_j) / 2) + low_j
    close_i = stock_data.iloc[i]['Close']
    high_i = stock_data.iloc[i]['High']
    low_i = stock_data.iloc[i]['Low']
    op_i = stock_data.iloc[i]['Open']
    close_j = stock_data.iloc[j]['Close']
    high_j = stock_data.iloc[j]['High']
    low_j = stock_data.iloc[j]['Low']
    op_j = stock_data.iloc[j]['Open']
    close_k = stock_data.iloc[k]['Close']
    high_k = stock_data.iloc[k]['High']
    low_k = stock_data.iloc[k]['Low']
    op_k = stock_data.iloc[k]['Open']

    # calculate the reference , that is minimum of all lows
    min_low = min(low_i, low_j, low_k)

    # high_i_rel = high_i / min_low
    # low_i_rel = low_i / min_low
    # high_j_rel = high_j / min_low
    # low_j_rel = low_j / min_low
    # high_k_rel = high_k / min_low
    # low_k_rel = low_k / min_low
    # open_i_rel = op_i / min_low
    # open_j_rel = op_j / min_low
    # open_k_rel = op_k / min_low
    # close_i_rel = close_i / min_low
    # close_j_rel = close_j / min_low
    # close_k_rel = close_k / min_low

    m1 = (((high_i - low_i) / 2) + (low_i-min_low)) / min_low
    m2 = (((high_j - low_j) / 2) + (low_j-min_low)) / min_low
    m3 = (((high_k - low_k) / 2) + (low_k-min_low)) / min_low

    m1_close = (((close_i - op_i) / 2) + (op_i-min_low) ) / min_low
    m2_close = (((close_j - op_j) / 2) + (op_j-min_low) ) / min_low
    m3_close = (((close_k - op_k) / 2) + (op_k-min_low) ) / min_low
    # return (high_i_rel, low_i_rel, high_j_rel, low_j_rel, high_k_rel, low_k_rel, open_i_rel, open_j_rel, open_k_rel, close_i_rel, close_j_rel, close_k_rel, m1, m2, m3,  m1_close, m2_close, m3_close )
    return ( m1, m2, m3,  m1_close, m2_close, m3_close)

In [10]:
class StockPattern():
    def __init__(self, i, patt, dir):
        self.pattern = patt
        self.i = i
        self.dir = dir
    def __getitem__(self, index):
        return self.pattern[index]

    def __len__(self):
        return len(self.pattern)


In [11]:
# patt_index = {}
patterns = []
threshold = 0.02
for i in range(len(stock_data)-6):
    up = False
    down = False
    # check if any of the value of high in the range i+3 to i+6 is 3% greater than i+2 close value
    for j in range(i+3, i+4):
        if stock_data.iloc[j]['High'] >= stock_data.iloc[i+2]['Close'] * (1+threshold):
            up=True
            break
        elif stock_data.iloc[j]['Low']<= stock_data.iloc[i+2]['Close'] * (1-threshold):
            down = True
            break
    if up or down:
        cs_t_2 = getRelativeValues(i,stock_data)
        cs_t_1 = getRelativeValues(i+1,stock_data)
        cs_t = getRelativeValues(i+2,stock_data)
        relative_features = calculateMidpoints(i,i+1,i+2,stock_data)

        if up :
            patterns.append(StockPattern(i,np.array([*cs_t_2,*cs_t_1,*cs_t,*relative_features]),1))
            # patterns.append(StockPattern(i,np.array([*relative_features]),1))

        if down:
            patterns.append(StockPattern(i,np.array([*cs_t_2,*cs_t_1,*cs_t,*relative_features]),-1))
            # patterns.append(StockPattern(i,np.array([*relative_features]),-1))


from collections import defaultdict
mp = defaultdict(int)
for pat in patterns:
    mp[pat.dir]+=1
mp

defaultdict(int, {-1: 68, 1: 157})

In [12]:
# cluster all the patterns using agglomerative clustering
from sklearn.cluster import AgglomerativeClustering



In [13]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

from sklearn.cluster import AgglomerativeClustering


def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)
    print(pd.DataFrame(linkage_matrix))

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


cluster = AgglomerativeClustering(compute_full_tree=True, n_clusters=None, linkage='average', distance_threshold=0)
model = cluster.fit(patterns)




In [14]:
# plt.figure(figsize=(24, 18))
# plt.title("Hierarchical Clustering Dendrogram")
# # plot the top three levels of the dendrogram
# plot_dendrogram(model, truncate_mode="level", )
# plt.xlabel("Number of points in node (or index of point if no parenthesis).")
# plt.show()

In [15]:
import itertools
ii = itertools.count(np.array(patterns).shape[0])
tree = {next(ii) : {'left': x[0], 'right':x[1]} for x in model.children_}
root = max(tree.keys())
class Node:
    def __init__(self, left, right,id):
        self.children = []
        self.left = left
        self.right = right
        self.id = id
new_tree = {}
def build_tree(node):
    if node not in tree:
        leaf = Node(None, None,node)
        leaf.children = [patterns[node]]
        return leaf
    left = build_tree(tree[node]['left'])
    right = build_tree(tree[node]['right'])
    new_node = Node(left, right,node)
    new_node.children = left.children + right.children
    new_tree[node] = new_node
    return new_node

root_node = build_tree(root)


In [16]:
def reliability(node):
    ups = len([x for x in node.children if x.dir == 1])
    down = len([x for x in node.children if x.dir == -1])
    most = max(ups, down)
    final_dir = "up" if ups == most else "down"

    return most/len(node.children) ,final_dir

In [17]:
# plot the pattern discovered

# plot_pattern is a function that takes and index i and stock_data and finally plots three candles from the index i
def plot_pattern(i, stock_data, direction):
    # plot the pattern discovered
    df = stock_data.iloc[i:i+3]
    plt.figure(figsize=(10/4,5/4))
    mpf.plot(df, type='candle', style='yahoo', title=direction, ylabel='Price',
         ylabel_lower='Volume', volume=True, figsize=(10/4,5/4))

# plot_pattern(100, stock_data, 'bullish')

In [18]:
all_patterns = {"up" : [], "down": []}
reliability_threshold = 0.75
repeatability_threshold = 15
def get_pattern(node):
    if node is None :
        return 0 ,0 , ""
    rel , dir = reliability(node)
    combine = 0
    rel_r , combine_r, dir_r = get_pattern(node.right)
    combine += combine_r
    if combine_r == 0 and (rel_r > reliability_threshold and len(node.right.children) >= repeatability_threshold) :

        all_patterns[dir_r].append(node.right)
    rel_l , combine_l, dir_l = get_pattern(node.left)
    combine += combine_l
    if combine_l == 0 and (rel_l > reliability_threshold and len(node.left.children) >= repeatability_threshold) :

        all_patterns[dir_l].append(node.left)
    return rel, combine, dir

get_pattern(root_node)
print("down",len(all_patterns["down"]),all_patterns["down"])
print("up",len(all_patterns["up"]),all_patterns["up"])



down 0 []
up 8 [<__main__.Node object at 0x00000220C2521210>, <__main__.Node object at 0x00000220C2521290>, <__main__.Node object at 0x00000220C2521310>, <__main__.Node object at 0x00000220C2521390>, <__main__.Node object at 0x00000220C2521410>, <__main__.Node object at 0x00000220C2521910>, <__main__.Node object at 0x00000220C251E710>, <__main__.Node object at 0x00000220C251CB50>]


In [19]:
for key, val in all_patterns.items():
    for i,node in enumerate(val):
        print(f"pattern {i} with children {len(node.children)}")
        start = node.children[0].pattern
        for x in range(1,len(node.children)):
            start = np.add(start,node.children[x].pattern)
        print(start/len(node.children))
        # for child in node.children:
        #     plot_pattern(child.i,stock_data,str(child.dir)+" "+key)

pattern 0 with children 21
[1.02551735 0.99811245 0.98111049 1.02068652 0.99453411 0.97878198
 1.01529596 0.99130758 0.97803069 0.02970693 0.02649746 0.0238115
 0.0253533  0.02398434 0.02278342]
pattern 1 with children 27
[1.02295743 0.9978187  0.9814184  1.01827292 0.99423118 0.97830197
 1.01974742 0.99651644 0.98105173 0.02821915 0.02550906 0.02688187
 0.02486705 0.02432416 0.02469295]
pattern 2 with children 37
[1.01901019 0.99278428 0.97671343 1.02023639 0.99698672 0.98108295
 1.01719773 0.99554757 0.9789363  0.02888106 0.02732188 0.02680878
 0.02739523 0.02510953 0.02652126]
pattern 3 with children 39
[1.02047972 0.99297468 0.97683649 1.02183788 0.99733182 0.98157135
 1.01732893 0.99577152 0.97915417 0.0294204  0.02768138 0.02669012
 0.0272153  0.02458574 0.02633695]
pattern 4 with children 45
[1.02004954 0.99345725 0.97811241 1.0244418  1.00025286 0.9820999
 1.01741357 0.99311024 0.9768319  0.02910834 0.02890008 0.02788971
 0.02671271 0.02569627 0.02732125]
pattern 5 with childre

In [20]:
# calculate mean vector from list of pattern vector
def mean_vector(vectors):
    return np.sum(vectors, axis=0) / len(vectors)

In [21]:
def distance(a, b):
    a_min_b = a - b
    return np.sqrt(np.sum(a_min_b ** 2))

In [48]:
mean_pat = {"up" : [], "down" : []}
for key, val in all_patterns.items():
    for i,node in enumerate(val):
        trend = 1 if key == "up" else -1
        pat = {}
        # mean_vec = mean_vector(node.children)
        
        print(f"______________=================== pattern {i} with children {len(node.children)} ================_______________")
        mean_vec = np.mean([stockPat for stockPat in node.children if stockPat.dir == trend], axis=0)
        pat["mean_vec"]=mean_vec
        distances = np.array([distance(vec.pattern, mean_vec) for vec in node.children])
        std = np.std(distances)
        mean = np.mean(distances)
        pat["std"]=std
        pat["mean"]=mean
        pat["threshold"]=mean+std
        pat["total"] = 0
        pat["success"] = 0
        mean_pat[key].append(pat)
        # for vec in node.children:

        #     if distace(vec.pattern, mean_vec) > mean+std:
        #         print(distace(vec.pattern, mean_vec))
        #         plot_pattern(vec.i,stock_data,str(vec.dir)+" "+key)


______________=================== pattern 0 with children 21 ================_______________
______________=================== pattern 1 with children 27 ================_______________
______________=================== pattern 2 with children 37 ================_______________
______________=================== pattern 3 with children 39 ================_______________
______________=================== pattern 4 with children 45 ================_______________
______________=================== pattern 5 with children 50 ================_______________
______________=================== pattern 6 with children 19 ================_______________
______________=================== pattern 7 with children 17 ================_______________


In [27]:
# patt_index = {}

patterns = []
threshold = 0.01
for i in range(len(stock_data)-6):

    cs_t_2 = getRelativeValues(i,stock_data)
    cs_t_1 = getRelativeValues(i+1,stock_data)
    cs_t = getRelativeValues(i+2,stock_data)
    relative_features = calculateMidpoints(i,i+1,i+2,stock_data)
    # check if any pattern is found. A pattern is found if the distance between the pattern is less than the mean pattern and its standard deviation.
    test_vec = np.array([*cs_t_2,*cs_t_1,*cs_t,*relative_features])
    # test_vec = np.array([*relative_features])

    for key in  mean_pat.keys():
        for pat in mean_pat[key]:
            if distance(pat["mean_vec"],test_vec) < pat["mean"] + pat["std"] *1.125:
                pat["total"]+=1
                # check if the market went up or down
                up = False
                down = False
                for j in range(i+3, i+4):
                    if stock_data.iloc[j]['High'] >= stock_data.iloc[i+2]['Close'] * (1+threshold):
                        up=True
                        break
                    elif stock_data.iloc[j]['Low']<= stock_data.iloc[i+2]['Close'] * (1-threshold):
                        down = True
                        break
                if (up and key == "up") or (down and key == "down") :
                    pat["success"]+=1
                    break



In [28]:
mean_pat

{'up': [{'mean_vec': array([1.02637401, 0.99789137, 0.98057794, 1.02205528, 0.99488844,
          0.98030055, 1.01528627, 0.9901182 , 0.97828252, 0.02965372,
          0.02690838, 0.02393262, 0.02502588, 0.02309268, 0.02216258]),
   'std': 0.007352616132989003,
   'mean': 0.03345912727983253,
   'threshold': 0.04081174341282153,
   'total': 28,
   'success': 25},
  {'mean_vec': array([1.02382382, 0.99762059, 0.98081024, 1.0193582 , 0.99436141,
          0.97945139, 1.01848188, 0.99539433, 0.98110673, 0.02853274,
          0.02607825, 0.02651303, 0.02495093, 0.02381252, 0.02435763]),
   'std': 0.008468315506225522,
   'mean': 0.035734238880208255,
   'threshold': 0.04420255438643378,
   'total': 13,
   'success': 10},
  {'mean_vec': array([1.01959659, 0.99331803, 0.97693276, 1.02058041, 0.99695121,
          0.98196518, 1.01681584, 0.99543907, 0.97944647, 0.02838599,
          0.02739909, 0.02687035, 0.02677174, 0.02453877, 0.02645398]),
   'std': 0.008604083113471018,
   'mean': 0.0383

In [49]:

for i in range(len(stock_data)-6):
    
    cs_t_2 = getRelativeValues(i,stock_data)
    cs_t_1 = getRelativeValues(i+1,stock_data)
    cs_t = getRelativeValues(i+2,stock_data)
    relative_features = calculateMidpoints(i,i+1,i+2,stock_data)
    # check if any pattern is found. A pattern is found if the distance between the pattern is less than the mean pattern and its standard deviation.
    test_vec = np.array([*cs_t_2,*cs_t_1,*cs_t,*relative_features])
    # test_vec = np.array([*relative_features])
    # test_vec = get_candle_vector(i,3,stock_data)
    
    for key in  mean_pat.keys():
        
        for x,pat in enumerate(mean_pat[key]):
            if "unSuccess" not in pat : pat["unSuccess"] = 0
            if distance(pat["mean_vec"],test_vec) < pat["mean"] + pat["std"] *1.5:
                pat["total"]+=1
                # check if the market went up or down
                up = False
                down = False
                threshold = stock_data.iloc[i+2]['pct_change']
                checkForUpTrend = True
                checkForDownTrend = True
                # if x==2 :
                #     plot_pattern(i,stock_data,key)
                # check if any of the value of high in the range i+3 to i+6 is 3% greater than i+2 close #value
                if key =="up":
                    for j in range(i+3, len(stock_data)):
                            lowerbound = stock_data.iloc[i+2]['Low'] - (threshold *2)
                            bottom_gap = stock_data.iloc[i+3]['Open'] - lowerbound
                            upperbound = stock_data.iloc[i+3]['Open'] + (bottom_gap * 1.5)
                            if checkForUpTrend and stock_data.iloc[j]['Low'] <= lowerbound:
                                checkForUpTrend = False
                                pat["unSuccess"]+=1
                                break
                            elif checkForUpTrend and  stock_data.iloc[j]['High'] >= upperbound:
                                up=True
                                checkForUpTrend = False
                                pat["success"]+=1
                                break
                else:
                    for j in range(i+3, len(stock_data)):
                            upperbound = stock_data.iloc[i+2]['High'] + threshold
                            top_gap = upperbound - stock_data.iloc[i+3]['Open']
                            lowerbound = stock_data.iloc[i+3]['Open'] - (top_gap * 1.5)
                            if  checkForDownTrend and stock_data.iloc[j]['High'] >= upperbound:
                                checkForDownTrend = False
                                pat["unSuccess"]+=1
                                break
                            elif checkForDownTrend and stock_data.iloc[j]['Low'] <= lowerbound:
                                down = True
                                pat["success"]+=1
                                
                                break
                    
                


In [50]:
mean_pat

{'up': [{'mean_vec': array([1.02637401, 0.99789137, 0.98057794, 1.02205528, 0.99488844,
          0.98030055, 1.01528627, 0.9901182 , 0.97828252, 0.02965372,
          0.02690838, 0.02393262, 0.02502588, 0.02309268, 0.02216258]),
   'std': 0.007352616132989003,
   'mean': 0.03345912727983253,
   'threshold': 0.04081174341282153,
   'total': 36,
   'success': 11,
   'unSuccess': 25},
  {'mean_vec': array([1.02382382, 0.99762059, 0.98081024, 1.0193582 , 0.99436141,
          0.97945139, 1.01848188, 0.99539433, 0.98110673, 0.02853274,
          0.02607825, 0.02651303, 0.02495093, 0.02381252, 0.02435763]),
   'std': 0.008468315506225522,
   'mean': 0.035734238880208255,
   'threshold': 0.04420255438643378,
   'total': 44,
   'success': 13,
   'unSuccess': 31},
  {'mean_vec': array([1.01959659, 0.99331803, 0.97693276, 1.02058041, 0.99695121,
          0.98196518, 1.01681584, 0.99543907, 0.97944647, 0.02838599,
          0.02739909, 0.02687035, 0.02677174, 0.02453877, 0.02645398]),
   'std':